In [1]:
import time

import numpy as np

import qcodes as qc

## Multidimensional scanning module
from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
)

## Dummy instruments for generating synthetic data
from qcodes.instrument_drivers.mock_instruments import (
    DummyChannelInstrument,
    DummyInstrument,
    DummyInstrumentWithMeasurement,
)
from qcodes.validators import Numbers

In [2]:
# A dummy signal generator with two parameters ch1 and ch2
dac = DummyInstrument("dac", gates=["ch1", "ch2"])

# A dummy digital multimeter that generates a synthetic data depending
# on the values set on the setter_instr, in this case the dummy dac
dmm = DummyInstrumentWithMeasurement("dmm", setter_instr=dac)

dci = DummyChannelInstrument("dci")

In [9]:
initialise_or_create_database_at("~/experiments_for_15_mins.db")

In [11]:
tutorial_exp = load_or_create_experiment(
    experiment_name="tutorial_exp", sample_name="synthetic data"
)

In [12]:
context_meas = Measurement(exp=tutorial_exp, name="context_example")

In [13]:
# Register the independent parameter...
context_meas.register_parameter(dac.ch1)
# ...then register the dependent parameter
context_meas.register_parameter(
    dci.A.dummy_parameter_with_setpoints, setpoints=(dac.ch1,)
)

In [14]:
dci.A.dummy_n_points.vals = Numbers(1, 1e7)
dci.A.dummy_n_points(10_000_000)
dci.A.dummy_parameter_with_setpoints().size

10000000

In [ ]:
# Time for periodic background database writes
# context_meas.write_period = 2
start = time.perf_counter()
with context_meas.run(in_memory_cache=False) as datasaver:
    for set_v in np.linspace(0, 100, 100):
        dac.ch1.set(set_v)
        get_v = dci.A.dummy_parameter_with_setpoints.get()
        datasaver.add_result(
            (dac.ch1, set_v), (dci.A.dummy_parameter_with_setpoints, get_v)
        )

    # Convenient to have for plotting and data access
    dataset = datasaver.dataset
stop = time.perf_counter()

print(f"Meas took {stop-start} s")

Starting experimental run with id: 16. 
Meas took 346.9017100998899


In [16]:
qc.config.dataset.export_chunked_export_of_large_files_enabled = True

start = time.perf_counter()
datasaver.dataset.export("netcdf", "foo.nc")
stop = time.perf_counter()

Large dataset detected. Will write to multiple files first and combine after, to reduce memory overhead.


Writing individual files:   0%|          | 0/100 [00:00<?, ?it/s]

Combining files:   0%|          | 0/302 [00:00<?, ?it/s]

In [19]:
qc.config.dataset.export_chunked_export_of_large_files_enabled = False

start = time.perf_counter()
datasaver.dataset.export("netcdf", "foo.nc")
stop = time.perf_counter()

MemoryError: Unable to allocate 14.9 GiB for an array with shape (1000000000, 2) and data type int64